In [2]:
include("datasetloader.jl")
include("benchmark.jl")
using LanguageIdentification
import LanguageIdentification as LI

In [3]:
@time LI.initialize()
last.(LI.vocabulary_sizes())|>mean

  1.501767 seconds (5.28 M allocations: 236.988 MiB, 5.14% gc time, 31.70% compilation time)


2515.54

In [7]:
typemin(Int):typemax(Int)|>typeof

UnitRange{Int64}

In [8]:
zero(UnitRange{Int64})

MethodError: MethodError: no method matching zero(::Type{UnitRange{Int64}})

Closest candidates are:
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period
   @ Dates /data/julia-1.9.3/share/julia/stdlib/v1.9/Dates/src/periods.jl:51
  zero(!Matched::T) where T<:Dates.TimeType
   @ Dates /data/julia-1.9.3/share/julia/stdlib/v1.9/Dates/src/types.jl:441
  zero(!Matched::LinearAlgebra.Diagonal)
   @ LinearAlgebra /data/julia-1.9.3/share/julia/stdlib/v1.9/LinearAlgebra/src/special.jl:374
  ...


In [17]:
HBS = TatoebaDataset("corpus/tatoeba", "corpus/tatoeba_test_index.txt", langs=["hbs"])


  0.113055 seconds (953.22 k allocations: 62.763 MiB, 5.79% gc time)


In [69]:
HBS = TatoebaDataset("corpus/tatoeba", "corpus/tatoeba_test_index.txt", langs=["hbs"])
@time for (x,y) in HBS
    d =  LI.count_all_ngrams(x)
    loglikelihood(d, d)
    
end

  0.116475 seconds (576.35 k allocations: 57.012 MiB, 6.28% gc time)


In [61]:
@time for (x,y) in HBS
    d =  LI.count_all_ngrams(x)
    loglikelihood(d, d)
    
end

  0.110466 seconds (576.35 k allocations: 57.012 MiB, 10.89% gc time)


In [34]:
const UNK = UInt8[]

UInt8[]

In [74]:
function loglikelihood(p_dict, logq_dict)
    sc = zero(valtype(p_dict))
    for (code, p) in p_dict
        if !haskey(logq_dict, code)
            code = UNK
        end
        logq = logq_dict[code]
        sc += p * logq
    end
    sc
end

loglikelihood (generic function with 1 method)

In [75]:
d =  LI.count_all_ngrams(HBS[1][1])
@code_warntype loglikelihood(d, d)

MethodInstance for loglikelihood(::Dict{Vector{UInt8}, Float32}, ::Dict{Vector{UInt8}, Float32})
  from loglikelihood(p_dict, logq_dict) @ Main /data/langid_expirement/benchmarks-tune.ipynb:1
Arguments
  #self#::Core.Const(loglikelihood)
  p_dict::Dict{Vector{UInt8}, Float32}
  logq_dict::Dict{Vector{UInt8}, Float32}
Locals
  @_4::Union{Nothing, Tuple{Pair{Vector{UInt8}, Float32}, Int64}}
  sc::Float32
  @_6::Int64
  p::Float32
  code::Vector{UInt8}
  logq::Float32
Body::Float32
1 ─       (sc = 0.0f0)
│   %2  = p_dict::Dict{Vector{UInt8}, Float32}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #6 if not %5
2 ┄       Core.NewvarNode(:(logq))
│   %8  = @_4::Tuple{Pair{Vector{UInt8}, Float32}, Int64}
│   %9  = Core.getfield(%8, 1)::Pair{Vector{UInt8}, Float32}
│   %10 = Base.indexed_iterate(%9, 1)::Core.PartialStruct(Tuple{Vector{UInt8}, Int64}, Any[Vector{UInt8}, Core.Const(2)])
│         (code = Core.getfield(%10, 1))

└──       goto #4 if not %17
3 ─       (code = Main.UNK)
4 ┄       (logq = Base.getindex(logq_dict, code))
│   %21 = sc::Float32
│   %22 = (p * logq)::Float32
│         (sc = %21 + %22)
│         (@_4 = Base.iterate(%2, %15))
│   %25 = (@_4 === nothing)::Bool
│   %26 = Base.not_int(%25)::Bool
└──       goto #6 if not %26
5 ─       goto #2
6 ┄       return sc



In [9]:
LI.initialize(ngram=[1,4])
sum(last.(LI.vocabulary_sizes()))

118988

In [10]:
P = LI.PROFILES[1]
cnts = zeros(7)
for (k, v) in P
    if !isempty(k)
        cnts[length(k)] += (v)
    end
end
cnts

7-element Vector{Float64}:
  -278.0685486793518
     0.0
     0.0
 -6886.424198389053
     0.0
     0.0
     0.0

In [11]:
maximum(exp.(maximum.(values.(LI.PROFILES))))

0.4469622f0

In [5]:
WV = WikiDataSet("corpus/wikipedia/test", langs=LANGS)
# TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)
;

In [10]:
Int

Int64

In [12]:
minimum(values(LI.PROFILES[findfirst(==("ces"), LI.LANGUAGES)]))

-14.554772f0

In [13]:
minimum(values(LI.PROFILES[findfirst(==("hbs"), LI.LANGUAGES)]))

-13.062822f0

In [7]:
HBS = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=["hbs"]);

In [14]:
HB = benchmark(
"LanguageIdentification.jl"=>t->langid(t),
 dataset=HBS, languages=["hbs"])

  0.524464 seconds (279.97 k allocations: 24.916 MiB, 3.17% gc time, 0.78% compilation time)


1-element Vector{Any}:
 "LanguageIdentification.jl" => [0.3082706766917293]

In [6]:
x,y = HBS[rand(1:length(HBS))]
langprob(x),x

(["eng" => 0.02915456457049254, "hau" => 0.029135362270912103, "ido" => 0.0291317835691766, "nld" => 0.02905863031655561, "spa" => 0.029047351798809736], "Мислим да је то фер.")

## wikipedia

In [10]:
WB = benchmark(
"LanguageIdentification.jl"=>t->langid(t),
 dataset=WV, languages=LANGS)

 59.977509 seconds (73.47 M allocations: 5.228 GiB, 1.33% gc time, 0.04% compilation time)


1-element Vector{Any}:
 "LanguageIdentification.jl" => [0.995, 0.995, 1.0, 0.99, 1.0, 0.965, 0.985, 0.965, 1.0, 1.0  …  1.0, 0.995, 0.985, 0.99, 0.985, 0.985, 1.0, 0.97, 0.985, 0.995]

In [11]:
showtable(WB, LANGS) # log norm sum norm within ngram

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.21% | 99.50% | 99.50% | 100.00% | 99.00% | 100.00% | 96.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 87.00% | 100.00% | 91.00% | 99.00% | 93.00% | 97.00% | 98.50% | 100.00% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 97.50% | 99.50% | 99.50% | 97.00% | 98.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 97.00% | 98.50% | 99.50% |


In [10]:
showtable(WB, LANGS) # log norm within ngram

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  97.86% | 99.50% | 99.50% | 100.00% | 98.50% | 100.00% | 97.00% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 72.00% | 100.00% | 89.50% | 99.00% | 91.50% | 95.00% | 98.50% | 100.00% | 98.50% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 97.00% | 99.50% | 99.50% | 99.50% | 97.50% | 96.50% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 98.50% | 98.50% | 98.50% | 100.00% | 97.00% | 98.50% | 99.50% |


In [9]:
showtable(WB, LANGS) # mean norm within ngram

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|------

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| LanguageIdentification.jl |  96.21% | 99.50% | 99.50% | 100.00% | 88.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% | 94.00% | 100.00% | 82.50% | 99.00% | 94.00% | 93.50% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 99.00% | 99.00% | 100.00% | 98.00% | 13.00% | 99.00% | 99.00% | 97.00% | 94.50% | 100.00% | 99.50% | 91.50% | 99.00% | 97.00% | 100.00% | 99.00% | 98.50% | 99.50% | 98.00% | 98.50% | 98.50% | 97.00% | 99.50% | 99.50% |


In [10]:
showtable(WB, LANGS) # mean norm within ngram

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.22% | 99.50% | 99.50% | 100.00% | 99.00% | 100.00% | 96.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 86.00% | 100.00% | 91.00% | 99.00% | 93.00% | 97.00% | 98.50% | 100.00% | 98.50% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 98.50% | 99.50% | 99.50% | 97.00% | 98.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 97.00% | 98.50% | 99.50% |


In [10]:
showtable(WB, LANGS) # init 4:4 & t->langid(t, ngram=4:4)

|                           | Average | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur     | lat    | lit     | mar    | mkd    | msa    | nds     | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|---------|--------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|---

|                           | Average | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur     | lat    | lit     | mar    | mkd    | msa    | nds     | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|---------|--------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.67% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 97.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 94.00% | 100.00% | 93.00% | 99.00% | 95.00% | 97.50% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 100.00% | 99.00% | 100.00% | 96.50% | 99.50% | 99.50% | 100.00% | 97.50% | 99.50% | 100.00% | 99.50% | 91.00% | 100.00% | 97.50% | 100.00% | 99.50% | 99.00% | 99.50% | 99.00% | 98.50% | 100.00% | 96.50% | 99.00% | 98.50% |


In [7]:
showtable(WB, LANGS) # t->langid(t, ngram=4:4)

|                           | Average | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur     | lat    | lit    | mar    | mkd    | msa    | nds     | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur     | lat    | lit    | mar    | mkd    | msa    | nds     | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  96.12% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 95.00% | 95.00% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 99.50% | 99.50% | 89.00% | 100.00% | 93.00% | 95.50% | 93.00% | 97.50% | 98.00% | 100.00% | 91.00% | 99.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 99.50% | 99.50% | 100.00% | 97.00% | 97.00% | 93.00% | 99.50% | 90.00% | 99.50% | 96.50% | 99.50% | 99.50% | 98.50% | 98.00% | 99.00% | 98.50% | 100.00% | 97.00% | 99.00% | 11.50% |


In [14]:
showtable(WB, LANGS) # weighted norm: *l^2

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo    | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|--------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|-----

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo    | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|--------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| LanguageIdentification.jl |  98.20% | 99.50% | 99.50% | 100.00% | 97.50% | 100.00% | 98.00% | 99.00% | 96.00% | 100.00% | 100.00% | 97.00% | 100.00% | 99.00% | 100.00% | 99.50% | 94.00% | 100.00% | 90.00% | 99.00% | 96.50% | 94.50% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.00% | 100.00% | 96.50% | 96.50% | 99.50% | 99.50% | 97.50% | 94.50% | 100.00% | 99.50% | 90.50% | 99.50% | 97.00% | 100.00% | 99.00% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% | 100.00% |


In [6]:
showtable(WB, LANGS) # weighted norm: *l

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|----

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.25% | 99.50% | 99.50% | 100.00% | 97.50% | 100.00% | 97.50% | 99.00% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.50% | 93.50% | 100.00% | 90.50% | 99.00% | 95.50% | 97.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.00% | 100.00% | 96.00% | 96.50% | 99.50% | 99.50% | 97.00% | 94.50% | 100.00% | 99.50% | 90.50% | 99.50% | 97.00% | 100.00% | 99.00% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% | 99.50% |


In [23]:
showtable(WB, LANGS) # cutoff=2, vocabulary=5000, ngram=4:4

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd     | msa    | nds     | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|-

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd     | msa    | nds     | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.71% | 100.00% | 100.00% | 100.00% | 100.00% | 100.00% | 97.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 95.50% | 100.00% | 93.00% | 99.00% | 95.00% | 97.50% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 99.00% | 100.00% | 96.50% | 100.00% | 99.50% | 100.00% | 97.50% | 99.50% | 100.00% | 99.50% | 91.00% | 100.00% | 97.50% | 100.00% | 99.50% | 99.00% | 99.50% | 99.50% | 99.00% | 100.00% | 96.50% | 98.50% | 98.50% |


In [20]:
showtable(WB, LANGS) # cutoff=2, vocabulary=5000, ngram=3:4

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd     | msa     | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd     | msa     | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| LanguageIdentification.jl |  98.55% | 100.00% | 100.00% | 100.00% | 100.00% | 100.00% | 97.00% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 93.50% | 100.00% | 92.50% | 98.50% | 95.00% | 97.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 99.00% | 100.00% | 96.00% | 100.00% | 100.00% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.50% | 100.00% | 97.50% | 99.50% | 99.00% | 98.50% | 99.50% | 98.50% | 99.00% | 100.00% | 96.50% | 98.50% | 100.00% |


In [12]:
showtable(WB, LANGS) # #vocabulary=1:5000, cutoff=2

|                           | Average    | ara        | bel         | ben         | bul         | cat         | ces        | dan        | deu        | ell         | eng         | epo         | fas         | fin        | fra         | hau        | hbs        | heb         | hin        | hun        | ido        | ina        | isl        | ita        | jpn        | kab        | kor         | kur        | lat        | lit         | mar        | mkd        | msa        | nds        | nld        | nor        | pol         | por        | ron        | rus         | slk        | spa        | swa        | swe        | tat        | tgl        | tur        | ukr         | vie        | yid        | zho        |
|---------------------------|------------|------------|-------------|-------------|-------------|-------------|------------|------------|------------|-------------|-------------|-------------|-------------|------------|-------------|------------|------------|-------------|------------|------

|                           | Average    | ara        | bel         | ben         | bul         | cat         | ces        | dan        | deu        | ell         | eng         | epo         | fas         | fin        | fra         | hau        | hbs        | heb         | hin        | hun        | ido        | ina        | isl        | ita        | jpn        | kab        | kor         | kur        | lat        | lit         | mar        | mkd        | msa        | nds        | nld        | nor        | pol         | por        | ron        | rus         | slk        | spa        | swa        | swe        | tat        | tgl        | tur        | ukr         | vie        | yid        | zho        |
|---------------------------|------------|------------|-------------|-------------|-------------|-------------|------------|------------|------------|-------------|-------------|-------------|-------------|------------|-------------|------------|------------|-------------|------------|------------|------------|------------|------------|------------|------------|------------|-------------|------------|------------|-------------|------------|------------|------------|------------|------------|------------|-------------|------------|------------|-------------|------------|------------|------------|------------|------------|------------|------------|-------------|------------|------------|------------|
| LanguageIdentification.jl | **98.31%** | **99.50%** | **100.00%** | **100.00%** | **100.00%** | **100.00%** | **96.50%** | **99.00%** | **96.50%** | **100.00%** | **100.00%** | **100.00%** | **100.00%** | **99.00%** | **100.00%** | **99.50%** | **87.00%** | **100.00%** | **91.00%** | **99.00%** | **94.50%** | **97.00%** | **98.50%** | **99.50%** | **98.00%** | **99.00%** | **100.00%** | **99.00%** | **98.50%** | **100.00%** | **95.50%** | **99.00%** | **99.50%** | **99.50%** | **97.50%** | **98.00%** | **100.00%** | **99.50%** | **90.50%** | **100.00%** | **97.00%** | **99.50%** | **99.00%** | **98.50%** | **99.00%** | **98.50%** | **99.00%** | **100.00%** | **97.00%** | **98.50%** | **99.50%** |


In [4]:
showtable(WB, LANGS) # default: cutoff=0.85, vocabulary=1000:5000

|                           | Average    | ara        | bel        | ben         | bul        | cat         | ces        | dan        | deu        | ell         | eng         | epo         | fas         | fin        | fra         | hau        | hbs        | heb         | hin        | hun        | ido        | ina        | isl        | ita         | jpn        | kab        | kor         | kur        | lat        | lit         | mar        | mkd        | msa        | nds        | nld        | nor        | pol         | por        | ron        | rus        | slk        | spa         | swa        | swe        | tat        | tgl        | tur        | ukr         | vie        | yid        | zho        |
|---------------------------|------------|------------|------------|-------------|------------|-------------|------------|------------|------------|-------------|-------------|-------------|-------------|------------|-------------|------------|------------|-------------|------------|---------

|                           | Average    | ara        | bel        | ben         | bul        | cat         | ces        | dan        | deu        | ell         | eng         | epo         | fas         | fin        | fra         | hau        | hbs        | heb         | hin        | hun        | ido        | ina        | isl        | ita         | jpn        | kab        | kor         | kur        | lat        | lit         | mar        | mkd        | msa        | nds        | nld        | nor        | pol         | por        | ron        | rus        | slk        | spa         | swa        | swe        | tat        | tgl        | tur        | ukr         | vie        | yid        | zho        |
|---------------------------|------------|------------|------------|-------------|------------|-------------|------------|------------|------------|-------------|-------------|-------------|-------------|------------|-------------|------------|------------|-------------|------------|------------|------------|------------|------------|-------------|------------|------------|-------------|------------|------------|-------------|------------|------------|------------|------------|------------|------------|-------------|------------|------------|------------|------------|-------------|------------|------------|------------|------------|------------|-------------|------------|------------|------------|
| LanguageIdentification.jl | **98.20%** | **99.50%** | **99.50%** | **100.00%** | **99.00%** | **100.00%** | **96.50%** | **98.50%** | **96.50%** | **100.00%** | **100.00%** | **100.00%** | **100.00%** | **99.50%** | **100.00%** | **99.50%** | **87.00%** | **100.00%** | **91.00%** | **99.00%** | **92.50%** | **97.00%** | **98.50%** | **100.00%** | **98.00%** | **99.00%** | **100.00%** | **99.00%** | **98.50%** | **100.00%** | **95.50%** | **97.50%** | **99.50%** | **99.50%** | **97.00%** | **98.00%** | **100.00%** | **99.50%** | **90.00%** | **99.50%** | **97.00%** | **100.00%** | **99.50%** | **98.50%** | **99.00%** | **98.50%** | **98.50%** | **100.00%** | **97.00%** | **98.50%** | **99.50%** |


In [22]:
showtable(WB, LANGS) #vocabulary=1:5000

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  97.98% | 99.50% | 99.50% | 100.00% | 98.00% | 100.00% | 96.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 94.00% | 100.00% | 88.50% | 99.00% | 92.50% | 97.00% | 98.50% | 100.00% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 91.00% | 88.00% | 99.50% | 99.50% | 97.00% | 98.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 97.00% | 98.50% | 99.00% |


In [10]:
showtable(WB, LANGS) #vocabulary=1000:5000

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.12% | 99.50% | 99.50% | 100.00% | 99.00% | 100.00% | 97.00% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% | 86.50% | 100.00% | 90.50% | 99.50% | 91.50% | 93.50% | 98.50% | 100.00% | 98.50% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 96.50% | 98.00% | 99.50% | 99.50% | 97.00% | 97.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 96.50% | 99.50% | 99.50% |


In [17]:
showtable(WB, LANGS) #vocabulary=2000:5000

|                           | Average | ara     | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|

|                           | Average | ara     | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.13% | 100.00% | 99.50% | 100.00% | 99.50% | 100.00% | 96.50% | 98.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 81.00% | 100.00% | 91.00% | 99.00% | 92.50% | 97.00% | 98.50% | 100.00% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 98.50% | 99.50% | 99.50% | 97.00% | 98.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.50% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 97.00% | 98.50% | 99.50% |


In [14]:
showtable(WB, LANGS) #vocabulary=5000:5000

|                           | Average | ara    | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|

|                           | Average | ara    | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.31% | 99.50% | 100.00% | 100.00% | 100.00% | 100.00% | 96.50% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.50% | 87.00% | 100.00% | 91.00% | 99.00% | 94.50% | 97.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 99.00% | 99.50% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.50% | 100.00% | 97.00% | 99.50% | 99.00% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 97.00% | 98.50% | 99.50% |


In [26]:
showtable(WB, LANGS) #vocabulary=1:5000, cutoff=0.9

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|------

|                           | Average | ara    | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  97.90% | 99.50% | 99.50% | 100.00% | 99.50% | 100.00% | 96.50% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.50% | 94.00% | 100.00% | 84.50% | 99.00% | 94.50% | 96.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 87.00% | 89.50% | 99.50% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.50% | 99.50% | 97.00% | 99.50% | 99.00% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 97.00% | 98.00% | 99.50% |


In [29]:
showtable(WB, LANGS) #vocabulary=1000:5000, cutoff=0.9

|                           | Average | ara    | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|----

|                           | Average | ara    | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|--------|---------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.37% | 99.50% | 100.00% | 100.00% | 99.00% | 100.00% | 96.50% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.50% | 94.00% | 100.00% | 91.00% | 99.00% | 94.50% | 96.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 97.50% | 99.50% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.50% | 99.50% | 97.00% | 99.50% | 99.00% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 97.00% | 98.50% | 99.50% |


In [34]:
showtable(WB, LANGS) #vocabulary=3500:3500, cutoff=0.9

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|----

|                           | Average | ara     | bel     | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita     | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|---------------------------|---------|---------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| LanguageIdentification.jl |  98.35% | 100.00% | 100.00% | 100.00% | 100.00% | 100.00% | 97.00% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.50% | 100.00% | 99.50% | 88.00% | 100.00% | 91.00% | 99.00% | 95.00% | 96.00% | 98.50% | 100.00% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 98.50% | 99.50% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.00% | 99.50% | 97.00% | 100.00% | 99.00% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 97.00% | 98.50% | 100.00% |


In [39]:
showtable(WB, LANGS) #vocabulary=2000:5000, cutoff=0.9

|                           | Average | ara     | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|----

|                           | Average | ara     | bel    | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------------------|---------|---------|--------|---------|--------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| LanguageIdentification.jl |  98.14% | 100.00% | 99.50% | 100.00% | 99.50% | 100.00% | 96.50% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.50% | 81.00% | 100.00% | 91.00% | 99.00% | 94.50% | 96.00% | 98.50% | 99.50% | 98.00% | 99.00% | 100.00% | 99.00% | 98.50% | 100.00% | 95.50% | 98.50% | 99.50% | 99.50% | 97.50% | 98.00% | 100.00% | 99.50% | 90.50% | 99.50% | 97.00% | 99.50% | 99.00% | 98.50% | 99.00% | 98.50% | 99.00% | 100.00% | 97.00% | 98.50% | 99.50% |


4246.64